In [7]:
import pandas as pd
import numpy as np

In [8]:
data = pd.read_csv('code_blocks_final_clean.csv')
data = data.drop(columns='Unnamed: 0', axis=1)
data.sample(3)

,code_block,tag
60221,df_annotations = pd.DataFrame(train_annotation...,converting json to required csv format
101346,'''Count total groups in variable Ticket.'''\n...,findings as expected name contains strings tha...
168819,\nretail_mv = retail.groupby(['CustomerID']).a...,['lets look at amount spend per customer reven...


In [9]:
data['tag'].nunique()

128808

In [10]:
data.shape

(221803, 2)

### Удаление стоп-слов, стемминг, оставляем по одному комментарию

In [11]:
from nltk import download
download('stopwords')

import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /home/kirill/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/kirill/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/kirill/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/kirill/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [12]:
from nltk.stem.snowball import SnowballStemmer 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

In [13]:
stemmer = SnowballStemmer('english')

In [18]:
st_wr = set(stopwords.words('english'))

Лемматайзер:

In [19]:
import nltk
from nltk.corpus import wordnet

wnl = WordNetLemmatizer()

# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(lemmatizer, sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

# example
lemmatize_sentence(wnl, 'kirill is playing with fire')

'kirill be play with fire'

Тут из нескольких комментариев выбирается самый короткий (обосновано тем, что тогда он с большей вероятностью встречается часто == больше похож на метку). Отсекаются пробелы и всякие пустые метки.

In [20]:
import re
def clear_tag(tag):
    ready_tag = tag.replace('_', ' ')
    ready_tag = re.sub('[^A-Za-z0-9 ]+', '', ready_tag)
    ready_tag = ready_tag.strip()                    # removes spaces in the start and in the end
    ready_tag = ready_tag.replace(' ', '_')
    return ready_tag

In [21]:
from tqdm import tqdm
all_to_del = set(['', "['']", ' ', '  ', "[' ']", '        ', '     ', '    ', '   ', "['', ' ']", '       ', '      ', \
             '             ', '         ', '          ', "']",\
                 "['   ", "['", "[' ", "['  ", "['    ", "['    "])
rows_to_del = []

In [22]:
for i in tqdm(range(data.shape[0])):
    my_string = data.loc[i, 'tag']
    
    if (my_string is not np.nan) and my_string is not None:
        
        min_len = len(data.loc[i, 'tag']) + 10
        min_el = None
        s = set(list(map(str, my_string.split("', '"))))
        
        for el in s:
            
            # getting rid of punctuation, spaces, adding "_"
            el = clear_tag(el)
            
            if el not in all_to_del and el is not None and len(el) < 100:
                
                # getting rid of stop words
                el = [w.lower() for w in el.split('_') if w.lower() not in st_wr]
                el = '_'.join(el)
                
                # stemming 
                el = stemmer.stem(el)
                
                # lemmatizing
                # el = lemmatize_sentence(wnl, el)
                
                if len(el) < min_len:
                    min_len = len(el)
                    min_el = el

        if min_el is not None:
            data.loc[i, 'tag'] = min_el
        else:
            rows_to_del.append(i)
    else:
        rows_to_del.append(i)

100%|██████████| 221803/221803 [00:53<00:00, 4167.51it/s]


Было данных:

In [23]:
data.shape

(221803, 2)

Осталось:

In [24]:
data = data.drop(rows_to_del)
data.shape

(178782, 2)

In [25]:
data = data.reset_index()
data.drop(columns=['index'], axis=1, inplace=True)

________

### Удаление пустых меток

In [26]:
data = data[data['tag'] != '']
data = data[data['tag'] != '_']
data = data[data['tag'] != '__']
data = data[data['tag'] != '___']
data = data[data['tag'] != '____']
data = data[data['tag'] != '_____']

### Оставим самые популярные метки

На данный момент обработки имеем уникаьных комментариев:

In [28]:
data['tag'].nunique()

70962

Оставим 5000 самых популярных

In [29]:
from collections import Counter

mc = Counter(data['tag']).most_common(5000)

In [30]:
strange_to_del = []

Если уникальных блоков кода с этим комментарием меньше 75, то удалим эту метку т.к. это скорее всего скопированный комментарий. Этот процесс проводится среди самых популярных 5000 тэгов.

In [31]:
for el in tqdm(mc):
    if data[data['tag']==el[0]]['code_block'].nunique() < 75:
        strange_to_del.append(el[0])

100%|██████████| 5000/5000 [00:37<00:00, 133.94it/s]


In [32]:
for el in tqdm(strange_to_del):
    data = data[data['tag'] != el]

100%|██████████| 4904/4904 [01:25<00:00, 57.40it/s]


Здесь теряется некоторое количество данных и остается:

In [33]:
print(data.shape)
print(data['tag'].nunique())

(118655, 2)
66058


Посмотрим на самые популярные 20 меток:

In [37]:
Counter(data['tag']).most_common(20)

[('specify_none_want_read_whole_fil', 2403),
 ('linear_algebra', 1859),
 ('model', 1223),
 ('check_answ', 714),
 ('predict', 683),
 ('train', 610),
 ('load_data', 518),
 ('plot', 505),
 ('visual', 493),
 ('preprocess', 458),
 ('code', 430),
 ('submiss', 405),
 ('feature_engin', 400),
 ('test', 393),
 ('random_forest', 391),
 ('eda', 342),
 ('import', 340),
 ('data_preprocess', 318),
 ('data_clean', 304),
 ('xgboost', 290)]

In [38]:
num_tags_to_drop = data['tag'].nunique() - 20
num_tags_to_drop

66038

Удаление самых непопулярные (nunique - 20) меток:

In [39]:
least_c_tags = set(Counter(data['tag']).most_common()[-num_tags_to_drop:])

In [40]:
for el in tqdm(least_c_tags):
    data = data[data['tag'] != el[0]]

100%|██████████| 66038/66038 [10:27<00:00, 105.32it/s]


Осталось:

In [47]:
print(data.shape)
print(data['tag'].nunique())

(13079, 2)
20


In [42]:
ready_data = \
data.reset_index().drop(columns=['index'], axis=1)

In [49]:
mc = Counter(ready_data['tag']).most_common()

In [50]:
#with open("most_common_50_1.txt", 'w') as output:
#    for row in mc:
#        output.write(str(row) + '\n')

На данном этапе метки вышли не самыми лучшими, поэтому давайте попробуем доработать их самостоятельно:

1) Вариант для стемминга (финальный)

In [51]:
ready_data['tag'].unique()

array(['plot', 'linear_algebra', 'model', 'check_answ', 'feature_engin',
       'submiss', 'preprocess', 'load_data', 'train', 'random_forest',
       'eda', 'predict', 'data_preprocess', 'xgboost', 'data_clean',
       'test', 'specify_none_want_read_whole_fil', 'code', 'import',
       'visual'], dtype=object)

In [52]:
jut_inds = ready_data[ready_data['tag'] == 'specify_none_want_read_whole_fil'].index
ready_data.drop(index=jut_inds, axis=0, inplace=True)
jut_inds = ready_data[ready_data['tag'] == 'create_x'].index
ready_data.drop(index=jut_inds, axis=0, inplace=True)
jut_inds = ready_data[ready_data['tag'] == 'uncom'].index
ready_data.drop(index=jut_inds, axis=0, inplace=True)



ready_data.loc[ready_data['tag'] == 'data_preprocessing', 'tag'] = 'preprocessing'
ready_data.loc[ready_data['tag'] == 'data_load', 'tag'] = 'load_data'
ready_data.loc[ready_data['tag'] == 'data', 'tag'] = 'load_data'
ready_data.loc[ready_data['tag'] == 'importing_data', 'tag'] = 'load_data'
ready_data.loc[ready_data['tag'] == 'reading_data', 'tag'] = 'load_data'
ready_data.loc[ready_data['tag'] == 'eda', 'tag'] = 'exploratory_data_analysi'
ready_data.loc[ready_data['tag'] == 'data_explor', 'tag'] = 'exploratory_data_analysi'



In [53]:
jut_inds = ready_data[ready_data['tag'] == 'code'].index
ready_data.drop(index=jut_inds, axis=0, inplace=True)
jut_inds = ready_data[ready_data['tag'] == '1'].index
ready_data.drop(index=jut_inds, axis=0, inplace=True)

ready_data.loc[ready_data['tag'] == 'importing_librari', 'tag'] = 'import_librari'
ready_data.loc[ready_data['tag'] == 'import', 'tag'] = 'import_librari'
ready_data.loc[ready_data['tag'] == 'librari', 'tag'] = 'import_librari'
ready_data.loc[ready_data['tag'] == 'training_model', 'tag'] = 'import_librari'


2) Вариант без стемминга, без лемматизации

In [55]:
#ready_data.loc[ready_data['tag'] == 'data_preprocessing', 'tag'] = 'preprocessing'
#ready_data.loc[ready_data['tag'] == 'modelling', 'tag'] = '
#ready_data.loc[ready_data['tag'] == 'model_building', 'tag'] = 'model_building'
#ready_data.loc[ready_data['tag'] == 'data_visualization', 'tag'] = 'visualization'
#ready_data.loc[ready_data['tag'] == 'data', 'tag'] = 'load_data'
#ready_data.loc[ready_data['tag'] == 'exploratory_data_analysis', 'tag'] = 'eda'
#ready_data.loc[ready_data['tag'] == 'training', 'tag'] = 'train'

#jut_inds = ready_data[ready_data['tag'] == 'just_uncomment_them'].index
#ready_data.drop(index=jut_inds, axis=0, inplace=True)
#jut_inds = ready_data[ready_data['tag'] == 'check_your_answer'].index
#ready_data.drop(index=jut_inds, axis=0, inplace=True)
#jut_inds = ready_data[ready_data['tag'] == 'check_your_answers'].index
#ready_data.drop(index=jut_inds, axis=0, inplace=True)
#jut_inds = ready_data[ready_data['tag'] == 'your_code_here'].index
#ready_data.drop(index=jut_inds, axis=0, inplace=True)

3) Вариант с использованием только лемматизации:

In [56]:
#ready_data.loc[ready_data['tag'] == 'check_your_answers', 'tag'] = 'check_your_answer'
#ready_data.loc[ready_data['tag'] == 'data_preprocessing', 'tag'] = 'preprocessing'
#ready_data.loc[ready_data['tag'] == 'training', 'tag'] = 'train'
#ready_data.loc[ready_data['tag'] == 'exploratory_data_analysis', 'tag'] = 'eda'
#ready_data.loc[ready_data['tag'] == 'data', 'tag'] = 'load_data'
#ready_data.loc[ready_data['tag'] == 'model_building', 'tag'] = 'model'

#jut_inds = ready_data[ready_data['tag'] == 'just_uncomment_them'].index
#ready_data.drop(index=jut_inds, axis=0, inplace=True)
#jut_inds = ready_data[ready_data['tag'] == 'your_code_here'].index
#ready_data.drop(index=jut_inds, axis=0, inplace=True)'''

Вариант для стемминга:

In [57]:
#ready_data.loc[ready_data['tag'] == 'data', 'tag'] = 'load_data'
#ready_data.loc[ready_data['tag'] == 'train_model', 'tag'] = 'train'
#ready_data.loc[ready_data['tag'] == 'fit_model', 'tag'] = 'train'
#ready_data.loc[ready_data['tag'] == 'data_preprocess', 'tag'] = 'preprocess'
#ready_data.loc[ready_data['tag'] == 'check_answers', 'tag'] = 'check_answer'
#ready_data.loc[ready_data['tag'] == 'model', 'tag'] = 'model_building'
#ready_data.loc[ready_data['tag'] == 'data_visu', 'tag'] = 'visual'
#ready_data.loc[ready_data['tag'] == 'model_building', 'tag'] = 'model_build'

#jut_inds = ready_data[ready_data['tag'] == 'uncomment_them'].index
#ready_data.drop(index=jut_inds, axis=0, inplace=True)'''

Результаты:

In [58]:
print(ready_data.shape)
print(ready_data['tag'].nunique())
Counter(ready_data['tag']).most_common()

(10246, 2)
18


[('linear_algebra', 1859),
 ('model', 1223),
 ('check_answ', 714),
 ('predict', 683),
 ('train', 610),
 ('load_data', 518),
 ('plot', 505),
 ('visual', 493),
 ('preprocess', 458),
 ('submiss', 405),
 ('feature_engin', 400),
 ('test', 393),
 ('random_forest', 391),
 ('exploratory_data_analysi', 342),
 ('import_librari', 340),
 ('data_preprocess', 318),
 ('data_clean', 304),
 ('xgboost', 290)]

Восстановление баланса классов:

In [59]:
todel_mc1 = np.random.choice(
    np.array(ready_data[ready_data['tag'] == 'linear_algebra'].index),
    1859 - 1200,
    replace = False
)

ready_data.drop(index=todel_mc1, inplace=True, axis=0)

In [60]:
print(ready_data.shape)
print(ready_data['tag'].nunique())
Counter(ready_data['tag']).most_common()

(9587, 2)
18


[('model', 1223),
 ('linear_algebra', 1200),
 ('check_answ', 714),
 ('predict', 683),
 ('train', 610),
 ('load_data', 518),
 ('plot', 505),
 ('visual', 493),
 ('preprocess', 458),
 ('submiss', 405),
 ('feature_engin', 400),
 ('test', 393),
 ('random_forest', 391),
 ('exploratory_data_analysi', 342),
 ('import_librari', 340),
 ('data_preprocess', 318),
 ('data_clean', 304),
 ('xgboost', 290)]

In [62]:
jut_inds = ready_data[ready_data['tag'] == 'check_answ'].index
ready_data.drop(index=jut_inds, axis=0, inplace=True)

In [63]:
print(ready_data.shape)
print(ready_data['tag'].nunique())
Counter(ready_data['tag']).most_common()

(8873, 2)
17


[('model', 1223),
 ('linear_algebra', 1200),
 ('predict', 683),
 ('train', 610),
 ('load_data', 518),
 ('plot', 505),
 ('visual', 493),
 ('preprocess', 458),
 ('submiss', 405),
 ('feature_engin', 400),
 ('test', 393),
 ('random_forest', 391),
 ('exploratory_data_analysi', 342),
 ('import_librari', 340),
 ('data_preprocess', 318),
 ('data_clean', 304),
 ('xgboost', 290)]

____________

Разделим на train, test, val в соотношении 8.5 / 1.5 / 1.5

Поделим нацело:

In [80]:
(ready_data.shape[0] - 7) / 11

804.0

In [81]:
ready_data = ready_data.drop(ready_data.tail(7).index)

То есть размер обучающей выборки:

In [82]:
ready_data.shape[0] / 11 * 8

6432.0

Разделение на train, test, validation с помощью StratifiedShuffleSplit

In [84]:
from sklearn.model_selection import StratifiedShuffleSplit
SEED = 42001

split = StratifiedShuffleSplit(n_splits=1, test_size=3/11, random_state=SEED)
for train_index, test_valid_index in split.split(\
                                                 ready_data.drop('tag', axis=1), \
                                                 ready_data['tag']\
                                                ):
    train_set = ready_data.iloc[train_index]
    test_valid_set = ready_data.iloc[test_valid_index]

split2 = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=SEED)
for test_index, valid_index in split2.split(test_valid_set, test_valid_set['tag']):
    test_set = test_valid_set.iloc[test_index]
    valid_set = test_valid_set.iloc[valid_index]

In [85]:
train_set = train_set.reset_index()
test_set = test_set.reset_index()
valid_set = valid_set.reset_index()

______

Далее все '\n' заменяются на '\n\t', а также вначале каждого сниппета приписывается

'def tag():\n\t'

Каждый сниппет записывается в **отдельный файл** с названием номер_итерации.py в каталог my_train, my_test или my_valid (требование к формату code2vec).

In [86]:
def add_tabs(exmpl):
    new_exmpl = str()
    for i in range(len(exmpl)):
        if (exmpl[i]=='\n'):
            new_exmpl += '\n\t'
        else:
            new_exmpl += exmpl[i]
    return new_exmpl

train:

In [158]:
for i in range(train_set.shape[0]):
        exmpl = train_set.loc[i]['code_block'][:-1]
        exmpl = add_tabs(exmpl)
        print('def ' + str(train_set.loc[i]['tag'])+'():\n\t' + exmpl
              ,  file=open('/home/kirill/code2vec/cd2vec/python/my_train/func'+str(i)+'.py', 'w'))

test:

In [159]:
for i in range(test_set.shape[0]):
        exmpl = test_set.loc[i]['code_block'][:-1]
        exmpl = add_tabs(exmpl)
        print('def ' + str(test_set.loc[i]['tag'])+'():\n\t' + exmpl
              ,  file=open('/home/kirill/code2vec/cd2vec/python/my_test/func'+str(i)+'.py', 'w'))

valid:

In [160]:
for i in range(valid_set.shape[0]):
        exmpl = valid_set.loc[i]['code_block'][:-1]
        exmpl = add_tabs(exmpl)
        print('def ' + str(valid_set.loc[i]['tag'])+'():\n\t' + exmpl
              ,  file=open('/home/kirill/code2vec/cd2vec/python/my_val/func'+str(i)+'.py', 'w'))